In [10]:
from transformers import AutoModel, AutoModelForSequenceClassification 
import torch
from typing import OrderedDict



In [11]:
def way1():
    ckpt = torch.load("results/IEMOCAP/roberta-large/final/2021-05-09-12-19-54-speaker_mode-upper-num_past_utterances-1000-num_future_utterances-0-batch_size-4-seed-4/checkpoint-5975/pytorch_model.bin",map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    bert = AutoModel.from_pretrained("roberta-large")
    new_state_dict = OrderedDict()
    for k, v in ckpt.items():
        name = k.replace("roberta." , "")  # remove 'roberta.' prefix
        
        new_state_dict[name] = v

    # Load the new state dictionary into your model
    bert.load_state_dict(new_state_dict , strict = False)
    # bert.load_state_dict(ckpt)
    # print(bert.classifier.out_proj.bias , flush = True)


    # Zero out all gradients. Might not need this anymore though
    # for i, model in enumerate([bert, whisper, videomae]):
    for i, model in enumerate([bert]):
        for param in model.parameters():
            param.requires_grad = False

    # Put the models onto eval mode
    bert.eval()
    return bert

In [ ]:
b = way1()

In [17]:
def way2():
    ckpt = torch.load("results/IEMOCAP/roberta-large/final/2021-05-09-12-19-54-speaker_mode-upper-num_past_utterances-1000-num_future_utterances-0-batch_size-4-seed-4/checkpoint-5975/pytorch_model.bin",map_location=torch.device("cuda" if torch.cuda.is_available() else "cpu"))
    bert = AutoModelForSequenceClassification.from_pretrained("roberta-large" , num_labels = 6)
    
    # Load the new state dictionary into your model
    bert.load_state_dict(ckpt , strict = False)
    # bert.load_state_dict(ckpt)
    # print(bert.classifier.out_proj.bias , flush = True)


    # Zero out all gradients. Might not need this anymore though
    # for i, model in enumerate([bert, whisper, videomae]):
    for i, model in enumerate([bert]):
        for param in model.parameters():
            param.requires_grad = False

    # Put the models onto eval mode
    bert.eval()
    return bert


In [18]:
c = way2()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
x = torch.randint(1 , 1000 ,(1 , 512))
x.shape

torch.Size([1, 512])

In [30]:
b(x)
last_hidden_states = b(x)['last_hidden_state']

In [29]:
hidden_states = c(x , output_hidden_states=True)['hidden_states']

In [33]:
last_hidden_states , last_hidden_states.shape

(tensor([[[-0.3282, -0.1843,  0.3596,  ..., -0.0457, -0.0643, -0.1438],
          [-0.2051, -0.2664,  0.1925,  ...,  0.0569, -0.0028, -0.1785],
          [-0.7907, -0.5648,  0.3171,  ..., -0.2565,  0.5877, -0.1158],
          ...,
          [-0.3575, -0.8840,  0.1654,  ...,  0.6791,  0.7535,  0.1315],
          [-0.2939, -0.2007,  0.3281,  ...,  0.7667,  0.8214, -0.4741],
          [-0.2771, -0.3241,  0.5546,  ...,  0.2184, -0.2696, -0.1907]]]),
 torch.Size([1, 512, 1024]))

In [35]:
hidden_states[0], hidden_states[0].shape

(tensor([[[ 0.2685, -0.8295,  0.3979,  ..., -0.5905, -0.4159, -0.6246],
          [-0.0218, -0.0437,  0.0613,  ..., -0.3169,  0.3718, -0.4067],
          [ 1.7885, -1.2825, -0.4770,  ..., -1.7930,  0.4633, -1.0925],
          ...,
          [ 0.2500,  0.6878,  1.1036,  ..., -0.6367, -0.6602, -0.2775],
          [ 0.1526, -0.4661,  0.0468,  ...,  0.8816,  0.3922,  0.1328],
          [-0.3057,  1.3320,  1.4506,  ...,  0.8307, -0.6729, -0.7277]]]),
 torch.Size([1, 512, 1024]))